In [ ]:
#| default_exp tui


# TUI - Terminal User Interface

Textual-based terminal user interface components for audio transcription.


In [ ]:
#| export
from enum import Enum, auto
import asyncio
import pyperclip

from textual import on
from textual.app import App, ComposeResult
from textual.reactive import reactive
from textual.containers import Container, Grid, Center
from textual.screen import ModalScreen
from textual.widgets import Header, Footer, Static, Button, Log, Select, Rule, MarkdownViewer

from tui_writer.live import LiveTranscriber
from tui_writer.ai import apply_instruction

In [ ]:
#| export

TEXTUAL_CSS = """
    Screen {
        align: center top;
    }
    Header {
        background: $background;
    }
    .recording {
        background: $error;
    }

    #transcript-display {
        hatch: horizontal $boost 80%;
        background: $boost;
        margin: 1 3;
        padding: 2 3;
    }
    
    HelpModal {
        align: center middle;
    }
    #help {
        border: thick $background 80%;
        background: $surface;
        width: 100;
        padding: 0 2;
        align: center top;
        height: 90%;
        max-height: 70;
    }

    
    SettingsModal {
        align: center middle;
    }
    #settings-modal {
        border: thick $background 80%;
        background: $surface;
        width: 100;
        padding: 0 2;
        align: center top;
        height: 20;
    }
    #modal-header {
        content-align: center middle;
        margin: 1 0;
    }
    Rule {
        margin: 0;
    }
    #settings-grid {
        layout: grid;
        grid-size: 3 3;
        grid-rows: 1fr;
        grid-columns: 1fr;
        grid-gutter: 1;
    }
    .select-text {
        height: 100%;
        content-align: center middle;
    }
    Select {
        column-span: 2;
    }
    #settings-actions {
        column-span: 3;
    }
"""

In [ ]:
#| export

class SettingsModal(ModalScreen):
    # Available Whisper model options (label, value)
    WHISPER_MODELS = [
        ("Tiny      ⚡  Ultra fast   - Low accuracy", "tiny"),
        ("Base      ⚡  Fast         - Decent accuracy", "base"),
        ("Small     ⚖   Balanced    - Good accuracy", "small"),
        ("Medium    🐢  Slow        - High accuracy",  "medium"),
        ("Large     🐢  Very Slow   - Best accuracy", "large-v3"),
        ("Tiny.en   English-only    - faster", "tiny.en"),
        ("Base.en   English-only    - faster", "base.en"),
        ("Small.en  English-only    - faster", "small.en"),
        ("Medium.en English-only    - faster", "medium.en"),
        ("Large.en  English-only    - faster", "large-v3.en")
    ]
    WHISPER_LANGUAGES = [
        ("English", "en"),
        ("Norwegian", "no"),
        ("Swedish", "sv"),
        ("Danish", "da"),
        ("Finnish", "fi"),
        ("German", "de"),
        ("Dutch", "nl"),
        ("French", "fr"),
        ("Spanish", "es"),
        ("Portuguese", "pt"),
        ("Italian", "it"),
        ("Polish", "pl"),
        ("Czech", "cs"),
        ("Slovak", "sk"),
        ("Hungarian", "hu"),
        ("Greek", "el"),
        ("Turkish", "tr"),
        ("Russian", "ru"),
        ("Ukrainian", "uk"),
        ("Arabic", "ar"),
        ("Hebrew", "he"),
        ("Hindi", "hi"),
        ("Bengali", "bn"),
        ("Urdu", "ur"),
        ("Persian (Farsi)", "fa"),
        ("Thai", "th"),
        ("Vietnamese", "vi"),
        ("Indonesian", "id"),
        ("Malay", "ms"),
        ("Filipino (Tagalog)", "tl"),
        ("Chinese (Mandarin)", "zh"),
        ("Chinese (Cantonese)", "yue"),
        ("Japanese", "ja"),
        ("Korean", "ko"),
        ("Swahili", "sw"),
        ("Afrikaans", "af"),
        ("Romanian", "ro"),
        ("Bulgarian", "bg"),
        ("Serbian", "sr"),
        ("Croatian", "hr"),
        ("Slovenian", "sl"),
        ("Estonian", "et"),
        ("Latvian", "lv"),
        ("Lithuanian", "lt"),
    ]

    def __init__(self, model, language):
        self.model = model
        self.language = language
        super().__init__()

    def compose(self) -> ComposeResult:
        with Container(id="settings-modal"):
            yield Static("SETTINGS", id="modal-header")
            yield Rule(line_style="heavy")
            with Grid(id="settings-grid"):
                yield Static("Transcribing model:", classes="select-text")
                yield Select(
                        options=self.WHISPER_MODELS,
                        allow_blank=False,
                        value=self.model,
                        id="whisper-model"
                )
                yield Static("Language:", classes="select-text")
                yield Select(
                        options=self.WHISPER_LANGUAGES,
                        allow_blank=False,
                        value=self.language,
                        id="whisper-language"
                )
                yield Center(
                    Button.success("Close (esc)", id="save"),
                    id="settings-actions",
                )


    # Whisper selectors
    @on(Select.Changed, "#whisper-model")
    def on_model_changed(self, event: Select.Changed) -> None:
        self.model = str(event.value)

    @on(Select.Changed, "#whisper-language")
    def on_language_changed(self, event: Select.Changed) -> None:
        self.language = str(event.value)

    def key_escape(self) -> None:
        self.dismiss([self.model, self.language])

    def on_button_pressed(self) -> None:
        self.dismiss([self.model, self.language])


In [ ]:
#| export
HELP_MARKDOWN = """\
# TUI Writer

Help screen on how to use and customize TUI Writer.

## How to use

### Live transcriber:

- Press SPACE on your keyboard, you should see the top bar turn red.
- Simply start speaking and you should see the transcriber in action.
- Stop transcribing by pressing SPACE again.

### Live editing:
If you want to change any of the transcribed text:

- Press 'e' to get started.
- Audibly tell the AI what edits you want.
- Press 'e' again to send the instruction to the AI.

If the edit did not go as planned:
- do this
"""

class HelpModal(ModalScreen):
    def compose(self) -> ComposeResult:
        with Container(id="help"):
            markdown_viewer = MarkdownViewer(HELP_MARKDOWN, show_table_of_contents=True)
            markdown_viewer.code_indent_guides = False
            yield markdown_viewer
            yield Center(Button("Close (esc)", variant="primary"))

    def on_button_pressed(self) -> None:
        self.app.pop_screen()

    def key_escape(self) -> None:
        self.app.pop_screen()

## TranscriptionTUI

Main terminal user interface application for audio transcription.

The TUI uses Textual's [theme system](https://textual.textualize.io/guide/design/) with built-in color variables like `$primary`, `$error`, `$success`, etc. 

**Available built-in themes:**
- `textual-dark` (default)
- `textual-light`
- `nord`
- `gruvbox`
- `tokyo-night`
- `solarized-light`

To change the theme, modify the `self.theme` assignment in `__init__`.


In [ ]:
#| export
# Constants for transcriber configuration
DEFAULT_VAD_THRESHOLD = 0.5
DEFAULT_MIN_SPEECH_DURATION_MS = 250
DEFAULT_MIN_SILENCE_DURATION_MS = 500
MIN_EDIT_INSTRUCTION_WORDS = 2

In [ ]:
#| export
class RecordingState(Enum):
    """High-level states the TUI can be in."""
    IDLE = auto()       # Not recording
    RECORDING = auto()  # Live transcription is running
    EDIT = auto()       # Voice-based editing mode

class TranscriptionTUI(App):
    """Terminal user interface for live audio transcription and AI-powered editing.
    
    Features:
    - Live speech-to-text transcription using Whisper
    - Voice-controlled text editing via AI
    - Configurable models and languages
    - Copy transcription to clipboard
    """

    # Class attributes
    CSS = TEXTUAL_CSS
    AUTO_FOCUS = False
    state = reactive(RecordingState.IDLE)
    BINDINGS = [
        ("q", "quit", "Quit"),
        ("?", "help_modal", "Help"),
        ("s", "settings_modal", "Settings"),
        ("space", "toggle_recording", "Start/Stop"),
        ("e", "make_edits", "Talk to edit"),
        ("c", "copy_transcription", "Copy transcript")
    ]

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # Transcription state
        self._transcription_task: asyncio.Task | None = None
        self._transcriber: LiveTranscriber | None = None
        self.full_transcript: str = ""
        self.edit_instructions: str = ""

        # Configuration
        self.whisper_model = "base"
        self.language = "en"

    def compose(self) -> ComposeResult:
        """Construct and layout all UI widgets."""
        yield Header(show_clock=True)
        yield Log(id="transcript-display", auto_scroll=True)
        yield Footer()


    # === UI Actions (triggered by key bindings) ===
    def action_help_modal(self) -> None:
        self.push_screen(HelpModal())

    def action_settings_modal(self) -> None:
        self.push_screen(SettingsModal(self.whisper_model, self.language), self.apply_settings)

    async def action_toggle_recording(self) -> None:
        """Spacebar: start if idle, stop if currently recording."""
        if self.state is RecordingState.IDLE:
            await self._start(self.on_transcript_chunk)
            self.state = RecordingState.RECORDING
        elif self.state is RecordingState.RECORDING:
            self.title = "STOPPING..."
            await self._stop()
            self.state = RecordingState.IDLE

    def action_copy_transcription(self) -> None:
        if self.full_transcript:
            pyperclip.copy(self.full_transcript)
            self.notify("Copied transcription", title="Copied to clipboard!")
        else:
            self.notify("Nothing to copy yet!", severity="warning", title="Oops!")

    def _is_valid_edit_instruction(self, instruction: str) -> bool:
        """Check if edit instruction meets minimum requirements."""
        return len(instruction.strip().split()) >= MIN_EDIT_INSTRUCTION_WORDS

    async def action_make_edits(self) -> None:
        if self.state is not RecordingState.EDIT:
            if not self.full_transcript:
                self.notify("No transcript available to edit", severity="warning", title="Nothing to Edit")
                return
            
            await self._start(self.on_transcript_edit)
            self.state = RecordingState.EDIT 
        else:
            # Leaving edit mode
            self.transcript_display.loading = True
            await self._stop()

            if not self._is_valid_edit_instruction(self.edit_instructions):
                self.notify("Edit instruction too short - need more detail", severity="warning", title="Invalid Edit")
            else:
                self.full_transcript, _ = apply_instruction(self.full_transcript, self.edit_instructions)
                self.transcript_display.clear()
                self.transcript_display.write_lines(self.full_transcript.splitlines(True))
            
            self.transcript_display.loading = False
            self.edit_instructions = ""
            self.state = RecordingState.IDLE

    # === Settings & Modal Callbacks ===
    def apply_settings(self, settings: list[str]) -> None:
        model, language = settings
        self.whisper_model = model
        self.language = language

    # === Transcription Lifecycle ===
    async def _start(self, on_transcript_callback) -> None:
        self._transcriber = LiveTranscriber(
            model_id=self.whisper_model,
            language=self.language,
            on_transcript=on_transcript_callback,
            vad_threshold=DEFAULT_VAD_THRESHOLD,
            min_speech_duration_ms=DEFAULT_MIN_SPEECH_DURATION_MS,
            min_silence_duration_ms=DEFAULT_MIN_SILENCE_DURATION_MS,
        )
        self._transcription_task = asyncio.create_task(self._transcriber.start())

    async def _stop(self) -> None:
        try:
            if self._transcriber:
                await self._transcriber.stop()
            if self._transcription_task:
                await self._transcription_task
        finally:
            self._transcriber = None
            self._transcription_task = None


    # === Event Handlers & Callbacks ===
    def _normalize_text(self, text: str) -> str:
        """Normalize and clean text input."""
        return (text or "").strip()

    def on_transcript_chunk(self, text: str) -> None:
        """Called whenever the transcriber produces a new text chunk."""
        text = self._normalize_text(text)
        if not text:
            return
        self.full_transcript += text + "\n"
        self.transcript_display.write_line(text)
    
    def on_transcript_edit(self, instruction: str) -> None:
        """Collects spoken edit instructions during edit mode."""
        instruction = self._normalize_text(instruction)
        if not instruction:
            return
        self.edit_instructions += ("\n" if self.edit_instructions else "") + instruction

    def watch_state(self, new_state: RecordingState) -> None:
        """Reactively update UI when recording state changes."""
        self.sub_title = ""

        match new_state:
            case RecordingState.IDLE:
                self.title = "○ STANDBY"
                self.header.remove_class("recording")
            case RecordingState.RECORDING:
                self.title = "● RECORDING"
                self.header.add_class("recording")
            case RecordingState.EDIT:
                self.title = "● TALK TO EDIT"
                self.sub_title = "press 'e' to apply the instructions"
                self.header.add_class("recording")

    # === Lifecycle Hooks ===
    def on_mount(self) -> None:
        """Initialize widget references and set titles."""
        self.title = "○ STANDBY"
        self.header = self.query_one(Header)
        self.transcript_display: Log = self.query_one("#transcript-display", Log)
        self.theme = "textual-dark"

    async def on_unmount(self) -> None:
        """Cleanly stop background tasks when app closes."""
        await self._stop()